In [1]:
import geopandas as gpd
import pandas as pd 
import os

In [54]:
co_22_dir = r'C:\Users\phwh9568\Data\ParcelAtlas\CO_2022'
mhp_path = os.path.join(co_22_dir,'COSTAR_mhps.gpkg')
counties_dir = os.path.join(co_22_dir,'Counties')

In [14]:
fipsCodes = pd.read_csv(os.path.join(r'C:\Users\phwh9568\Data\ParcelAtlas', 'fips-by-state.csv'), dtype={'fips':str}, encoding='iso-8859-1')

In [12]:
mhps

,property_id,propertyname,property_address,propertycity,propertystate,propertycounty,propertyzipcode,propertystreetname,propertystreetnumber,propertystreetpostdirection,propertystreetpredirection,propertyzip5,latitude,longitude,parcelnumber1min,numberofunits,geometry
0,100 Ute Ave 80117,Ute Village Mobile Home Park,100 Ute Ave,Kiowa,CO,Elbert,80117,Ute Ave,100,None,None,80117,39.3493,-104.461,R107360,78.0,POINT (-104.46100 39.34930)
1,10110 E 120th Ave 80640,None,10110 E 120th Ave,Henderson,CO,Adams,80640-9390,120th Ave,10110,None,E,80640,39.9140,-104.870,1721-03-0-00-005,30.0,POINT (-104.87000 39.91400)
2,10201 Riverdale Rd 80229,Pine Lakes Ranch,10201 Riverdale Rd,Thornton,CO,Adams,80229,Riverdale Rd,10201,None,None,80229,39.8803,-104.935,1721-18-0-00-026,758.0,POINT (-104.93500 39.88030)
3,10210 Riverdale Rd 80229,Pine Lakes Ranch,10210 Riverdale Rd,Thornton,CO,Adams,80229-3115,Riverdale Rd,10210,None,None,80229,39.8800,-104.935,1721-18-0-00-026,1900.0,POINT (-104.93500 39.88000)
4,103 Coffman St 80501,None,103 Coffman St,Longmont,CO,Boulder,80501-5932,Coffman St,103,None,None,80501,40.1573,-105.105,1315102-00-019,56.0,POINT (-105.10500 40.15730)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,900 Mountain View Ave 80501,Mountain View MHC,900 Mountain View Ave,Longmont,CO,Boulder,80501-2734,Mountain View Ave,900,None,None,80501,40.1817,-105.106,1205342-38-001,81.0,POINT (-105.10600 40.18170)
155,9595 Pecos St 80260,Redwood Estates,9595 Pecos St,Thornton,CO,Adams,80260-5925,Pecos St,9595,None,None,80260,39.8716,-105.011,1719-21-2-00-006,754.0,POINT (-105.01100 39.87160)
156,9850 Federal Blvd 80260,Countryside Village,9850 Federal Blvd,Denver,CO,Adams,80260-5788,Federal Blvd,9850,None,None,80260,39.8740,-105.020,1719-17-4-00-003,345.0,POINT (-105.02000 39.87400)
157,Cobre Pl 81635,Battlement Mesa RV Park,Cobre Pl,Parachute,CO,Garfield,81635,Cobre Pl,None,None,None,81635,39.4239,-108.053,R270229,NaN,POINT (-108.05300 39.42390)


In [15]:
def mhomes_prepper(mhomesPath):
   mhomes = gpd.read_file(mhomesPath, layer='COSTAR_mhps')
   mhomes.sindex
   #columns = ['MHPID', 'NAME','ADDRESS', 'CITY', 'STATE', 'ZIP', 'STATUS', 'COUNTYFIPS', 'UNITS', 'SIZE', 'LATITUDE', 'LONGITUDE','geometry']
   #renames = ['MHPID', 'MH_NAME','MH_ADDRESS', 'MH_CITY', 'MH_STATE', 'MH_ZIP', 'MH_STATUS','MH_COUNTY_FIPS', 'MH_UNITS', 'MH_SIZE', 'MH_LATITUDE', 'MH_LONGITUDE']
   columns = ['property_id', 'propertyname', 'propertycity','propertystate', 'propertycounty', 'propertyzipcode', 'latitude', 'longitude', 'parcelnumber1min','numberofunits', 'geometry']
   renames = ['MH_prop_id', 'MH_prop_name', 'MH_prop_city', 'MH_prop_state', 'MH_prop_county', 'MH_prop_zip', 'MH_lat', 'MH_long', 'MH_parcel_num', 'MH_units']
   drops = [c for c in mhomes.columns if c not in columns] 
   renames = dict(zip(columns,renames))
   mhomes.drop(drops,axis=1, inplace=True)
   mhomes.rename(renames, axis='columns',inplace=True)
   return mhomes

In [18]:
mhps = mhomes_prepper(mhp_path)

In [19]:
mhps

,MH_prop_id,MH_prop_name,MH_prop_city,MH_prop_state,MH_prop_county,MH_prop_zip,MH_lat,MH_long,MH_parcel_num,MH_units,geometry
0,100 Ute Ave 80117,Ute Village Mobile Home Park,Kiowa,CO,Elbert,80117,39.3493,-104.461,R107360,78.0,POINT (-104.46100 39.34930)
1,10110 E 120th Ave 80640,None,Henderson,CO,Adams,80640-9390,39.9140,-104.870,1721-03-0-00-005,30.0,POINT (-104.87000 39.91400)
2,10201 Riverdale Rd 80229,Pine Lakes Ranch,Thornton,CO,Adams,80229,39.8803,-104.935,1721-18-0-00-026,758.0,POINT (-104.93500 39.88030)
3,10210 Riverdale Rd 80229,Pine Lakes Ranch,Thornton,CO,Adams,80229-3115,39.8800,-104.935,1721-18-0-00-026,1900.0,POINT (-104.93500 39.88000)
4,103 Coffman St 80501,None,Longmont,CO,Boulder,80501-5932,40.1573,-105.105,1315102-00-019,56.0,POINT (-105.10500 40.15730)
...,...,...,...,...,...,...,...,...,...,...,...
154,900 Mountain View Ave 80501,Mountain View MHC,Longmont,CO,Boulder,80501-2734,40.1817,-105.106,1205342-38-001,81.0,POINT (-105.10600 40.18170)
155,9595 Pecos St 80260,Redwood Estates,Thornton,CO,Adams,80260-5925,39.8716,-105.011,1719-21-2-00-006,754.0,POINT (-105.01100 39.87160)
156,9850 Federal Blvd 80260,Countryside Village,Denver,CO,Adams,80260-5788,39.8740,-105.020,1719-17-4-00-003,345.0,POINT (-105.02000 39.87400)
157,Cobre Pl 81635,Battlement Mesa RV Park,Parachute,CO,Garfield,81635,39.4239,-108.053,R270229,NaN,POINT (-108.05300 39.42390)


In [22]:
CO_fips = fipsCodes.loc[fipsCodes['fips'].str.startswith('08')].copy()

In [44]:
CO_fips['MH_prop_county'] = CO_fips['name'].str.replace(' County','')

In [45]:
CO_fips

,fips,name,state,MH_prop_county
244,08001,Adams County,CO,Adams
245,08003,Alamosa County,CO,Alamosa
246,08005,Arapahoe County,CO,Arapahoe
247,08007,Archuleta County,CO,Archuleta
248,08009,Baca County,CO,Baca
...,...,...,...,...
303,08117,Summit County,CO,Summit
304,08119,Teller County,CO,Teller
305,08121,Washington County,CO,Washington
306,08123,Weld County,CO,Weld


In [47]:
mhps = pd.merge(mhps,CO_fips[['MH_prop_county','fips',]], on='MH_prop_county')

In [52]:
mhps.rename({'fips':'MH_COUNTY_FIPS'}, axis='columns', inplace=True)

In [50]:
co_fips = mhps['fips'].unique().tolist()

In [55]:
for fips in co_fips:
    countyMHPs = mhps.loc[mhps['MH_COUNTY_FIPS'] == fips].copy()
    path = os.path.join(counties_dir,fips)
    countyMHPs.to_file(os.path.join(path,fips+'_COSTAR_mhps.gpkg'), driver='GPKG', layer='COSTAR_mhps')
    